<h1> 1 Installing Libraries </h1>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import os
import gc
import json
import pickle
import glob

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import os
import gc

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from sklearn.manifold import TSNE

from os import path
from PIL import Image

In [2]:
glove_path = 'E:\All Video\MLDL\glove'

In [3]:
def load_txt(name):
    with open(name, "r") as f:
        return f.read()

def load_pickle(name):
    with open(name, "rb") as handle:
        return pickle.load(handle)
    
def dump_pickle(name, file):
    with open(name, "wb") as handle:
        pickle.dump(file, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
drug_train_data = pd.read_csv("drug_preprocess.csv", index_col="Unnamed: 0")
drug_train_data.head(2)

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,review_len,review_n_words,review_by_patient_filter,review_by_patient_unique_word,len_review_by_patient_unique_word
10091,223808,Depakote ER,Epilepsy,"""After trying 4 different meds this one was th...",8,1-Apr-08,15,7.633384,370,74,after trying 4 different meds this one was the...,"['one', 'gladly', 'stayed', 'scary', 'well', '...",35
1009,35602,Anexsia,Pain,"""Worked better than Tylenol w/codeine #3 and C...",10,1-Apr-08,19,6.127475,88,14,worked better than tylenol w codeine 3 and ca...,"['w', 'tylenol', '3', 'better', 'pain', 'caris...",10


In [5]:
# To get the results in 4 decemal points
SAFE_DIV = 0.0001 

STOP_WORDS = stopwords.words("english")
stopword = set(STOP_WORDS)
stopword.add("said")
stopword.add("br")
stopword.add(" ")
stopword.remove("not")
stopword.remove("no")


def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will").replace("asap", "as soon as possible")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    
    porter = PorterStemmer()
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    
    
    if type(x) == type(''):
        x = porter.stem(x)
        example1 = BeautifulSoup(x)
        x = example1.get_text()
               
    return x
def combine_name_use_review(a, b, c):
    return preprocess(" ".join([a, b, c])) 
     

<h1>2 Combine Name Use case and Review </h1>

In [13]:
drug_train_data["combine_name_use_review"] = drug_train_data.loc[:, ["name_of_drug", "use_case_for_drug", "review_by_patient_filter"]].fillna("").apply(lambda x: combine_name_use_review(x["name_of_drug"], x["use_case_for_drug"], x["review_by_patient_filter"]), axis=1)
drug_train_data.head(2)

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,review_len,review_n_words,review_by_patient_filter,review_by_patient_unique_word,len_review_by_patient_unique_word,combine_name_use_review
10091,223808,Depakote ER,Epilepsy,"""After trying 4 different meds this one was th...",8,1-Apr-08,15,7.633384,370,74,after trying 4 different meds this one was the...,"['one', 'gladly', 'stayed', 'scary', 'well', '...",35,depakote er epilepsy after trying 4 different ...
1009,35602,Anexsia,Pain,"""Worked better than Tylenol w/codeine #3 and C...",10,1-Apr-08,19,6.127475,88,14,worked better than tylenol w codeine 3 and ca...,"['w', 'tylenol', '3', 'better', 'pain', 'caris...",10,anexsia pain worked better than tylenol w code...


In [14]:
all_train_name_use_review = list(drug_train_data["combine_name_use_review"])

In [15]:
def create_vocabulary():       # no use
    vocabulary = set()
    [vocabulary.update(sent.split()) for sent in drug_train_data["combine_name_use_review"].values]
    return vocabulary
vocabulary = create_vocabulary()
print(f"vocabulary contains {len(vocabulary)} words")

vocabulary contains 28094 words


<h1>3 Create vocab of most frequent words </h1>

In [16]:
def create_vocab(word_count_threshold = 10):
    word_counts = {}
    nsents = 0
    for sent in all_train_name_use_review:
        nsents += 1
        for w in sent.split(' '):
            word_counts[w] = word_counts.get(w, 0) + 1
    c = 0
    for key, val in word_counts.items():
        if len(key) == 1:
            c += 1
            print(key)
    print(c)
    vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
    return vocab, word_counts
vocab, word_counts = create_vocab(50)
print('Vocabulary = %d in our trainig data' % (len(vocab)))    # 10 -> 5607, 7-> 6705, 15-> 4477, 30->2985, 50 -> 

4
i
a
w
3
m
2
5
p
s
t
v
6
1
9
e
g
7
b
d
0
8
l
c
u
h
x
q
n
f
o
r
z
y
k
j
_
і
38
Vocabulary = 3164 in our trainig data


In [17]:
def ixtoword_wordtoix():
    ixtoword = {}
    wordtoix = {}
    ix = 1
    for w in vocab:
        wordtoix[w] = ix
        ixtoword[ix] = w
        ix += 1
    return ixtoword, wordtoix
if 'wordtoix.pkl' in os.listdir("../ix_word"):
    print("load")
    ixtoword = load_pickle('../ix_word/ixtoword.pkl')
    wordtoix = load_pickle('../ix_word/wordtoix.pkl')
else:
    ixtoword, wordtoix = ixtoword_wordtoix()
    dump_pickle('../ix_word/ixtoword.pkl', ixtoword)
    dump_pickle('../ix_word/wordtoix.pkl', wordtoix)
    print("dump")
vocab_size = len(ixtoword) + 1
print('size of vocabulary = %d in our trainig data' % (vocab_size))

load
size of vocabulary = 3167 in our trainig data


<h1>4 Maximum length of caption </h1>

In [18]:
max_length = max(len(d.split()) for d in all_train_name_use_review)
max_length

650

<h1>5 Embadding index and matrix </h1>

In [19]:
def create_embadding_index():
    embeddings_index = {} 
    f = open(os.path.join(glove_path, 'glove.6B.200d.txt'), encoding="utf-8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    return embeddings_index
if "embeddings_index.pkl" in os.listdir("../ix_word"):
    print('load')
    embeddings_index = load_pickle(os.path.join("../ix_word", "embeddings_index.pkl"))
else:
    print("dump")
    embeddings_index = create_embadding_index()
    dump_pickle(os.path.join("../ix_word", "embeddings_index.pkl"), embeddings_index)

print(f"There are {len(embeddings_index)} no of words in glove")

load


EOFError: Ran out of input

In [ ]:
def create_embadding_index():
    embeddings_index = {} 
    f = open(os.path.join(glove_path, 'glove.6B.200d.txt'), encoding="utf-8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    return embeddings_index
if "embeddings_index.pickle" in os.listdir(r"D:\computer vision\image_caption"):
    print('load')
    embeddings_index = load_pickle(os.path.join(r"D:\computer vision\image_caption", "embeddings_index.pickle"))
else:
    print("dump")
    embeddings_index = create_embadding_index()
    dump_pickle(os.path.join("../ix_word", "embeddings_index.pkl"), embeddings_index)

print(f"There are {len(embeddings_index)} no of words in glove")

In [20]:
def create_embadding_matrix():
    embedding_dim = 200
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in wordtoix.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            print(i, word)
    return embedding_matrix
if "embeddings_glove.pkl" in os.listdir("../ix_word"):
    print('load')
    embeddings_glove = load_pickle(os.path.join("../ix_word", "embeddings_glove.pkl"))
else:
    print("dump")
    embeddings_glove = create_embadding_matrix()
    dump_pickle(os.path.join("../ix_word", "embeddings_glove.pkl"), embeddings_glove)

print(f"The shape of glove representation is {embeddings_glove.shape}")

load
The shape of glove representation is (3167, 200)


<h1>6 DistilBert Embedding </h1>

In [21]:
# Refer: https://huggingface.co/transformers/model_doc/distilbert.html#

from transformers import DistilBertTokenizer, TFDistilBertModel

distil_bert = 'distilbert-base-uncased' # Name of the pretrained models

#DistilBERT 
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert)
model1 = TFDistilBertModel.from_pretrained(distil_bert)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [22]:
def find_vec(sent):
    vec = tokenizer.encode(sent)
    vec = tf.constant(vec)[None, :]
    return model1(vec)[0][0, 0, :].numpy()

# distill_bert_embedding = np.zeros((len(all_train_name_use_review), 768))
def create_embadding_distil():
    global distill_bert_embedding
    embedding_dim = 768
    for i, sent in enumerate(all_train_name_use_review):
        embedding_vector = find_vec(sent[:2000])
        print(i, sent[0:20])
        if embedding_vector is not None:
            distill_bert_embedding[i] = embedding_vector
        else:
            print(i,"NNNNNNNNNNNNNNNNNNNN",  word)
    return distill_bert_embedding

In [23]:
if 'distill_bert_embedding.pkl' in os.listdir("../ix_word"):
    print("load")
    distill_bert_embedding = load_pickle('../ix_word/distill_bert_embedding.pkl')
else:
    distill_bert_embedding = create_embadding_distil()
    dump_pickle('../ix_word/distill_bert_embedding.pkl', distill_bert_embedding)
    print("dump")

load


In [24]:
distill_bert_embedding.shape

(32165, 768)

In [25]:
combine_name_use_review_df = pd.DataFrame(distill_bert_embedding, columns=[f"review_{i}" for i in range(768)])
combine_name_use_review_df.head(2)

,review_0,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,...,review_758,review_759,review_760,review_761,review_762,review_763,review_764,review_765,review_766,review_767
0,0.059691,0.008081,0.080178,-0.266999,-0.264136,-0.360968,0.64633,0.423532,-0.074173,-0.070809,...,-0.140880,-0.184657,0.068444,-0.305117,0.219165,-0.065599,-0.351725,-0.171581,0.362696,-0.022024
1,-0.477708,-0.197286,-0.113292,-0.019980,-0.573079,-0.220722,0.34921,-0.051921,-0.013572,-0.067297,...,0.134501,-0.330719,-0.052216,0.190429,0.398500,0.230648,-0.213165,-0.396574,0.421274,0.357504


In [26]:
drug_train_data.head(2)

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,review_len,review_n_words,review_by_patient_filter,review_by_patient_unique_word,len_review_by_patient_unique_word,combine_name_use_review
10091,223808,Depakote ER,Epilepsy,"""After trying 4 different meds this one was th...",8,1-Apr-08,15,7.633384,370,74,after trying 4 different meds this one was the...,"['one', 'gladly', 'stayed', 'scary', 'well', '...",35,depakote er epilepsy after trying 4 different ...
1009,35602,Anexsia,Pain,"""Worked better than Tylenol w/codeine #3 and C...",10,1-Apr-08,19,6.127475,88,14,worked better than tylenol w codeine 3 and ca...,"['w', 'tylenol', '3', 'better', 'pain', 'caris...",10,anexsia pain worked better than tylenol w code...


In [27]:
print("Number of features in preprocessed dataframe :", drug_train_data.shape[1])
print("Number of features in nlp dataframe :", combine_name_use_review_df.shape[1])

Number of features in preprocessed dataframe : 14
Number of features in nlp dataframe : 768


<h1>7 Convert DataFrame to CSV File </h1>

In [28]:
# storing the final features to csv file
if not os.path.isfile('../ix_word/final_features.csv'):
    combine_name_use_review_df['patient_id']=drug_train_data['patient_id']
    combine_name_use_review_df['effectiveness_rating']=drug_train_data['effectiveness_rating']
    combine_name_use_review_df['drug_approved_by_UIC']=drug_train_data['drug_approved_by_UIC']
    combine_name_use_review_df['number_of_times_prescribed']=drug_train_data['number_of_times_prescribed']
    combine_name_use_review_df['len_review_by_patient_unique_word']=drug_train_data['len_review_by_patient_unique_word']
    combine_name_use_review_df['base_score']=drug_train_data['base_score']
    first_column = combine_name_use_review_df.pop('patient_id')
    combine_name_use_review_df.insert(0, 'patient_id', first_column)
    combine_name_use_review_df.to_csv('../ix_word/final_features.csv')

In [29]:
combine_name_use_review_df.head(2)

,review_0,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,...,review_758,review_759,review_760,review_761,review_762,review_763,review_764,review_765,review_766,review_767
0,0.059691,0.008081,0.080178,-0.266999,-0.264136,-0.360968,0.64633,0.423532,-0.074173,-0.070809,...,-0.140880,-0.184657,0.068444,-0.305117,0.219165,-0.065599,-0.351725,-0.171581,0.362696,-0.022024
1,-0.477708,-0.197286,-0.113292,-0.019980,-0.573079,-0.220722,0.34921,-0.051921,-0.013572,-0.067297,...,0.134501,-0.330719,-0.052216,0.190429,0.398500,0.230648,-0.213165,-0.396574,0.421274,0.357504


In [30]:
combine_name_use_review_df.shape

(32165, 768)

<h1>8 Test Data</h1>

In [31]:
drug_test_data = pd.read_csv("drug_test_preprocess.csv", index_col="Unnamed: 0")
drug_test_data.head(2)

,patient_id,name_of_drug,review_by_patient,drug_approved_by_UIC,number_of_times_prescribed,use_case_for_drug,effectiveness_rating,review_by_patient_filter,review_by_patient_unique_word,len_review_by_patient_unique_word,review_len,review_n_words
0,163740,Mirtazapine,"""I&#039;ve tried a few antidepressants over th...",28-Feb-12,22,Depression,10,i 039 ve tried a few antidepressants over the...,"['insomnia', 'changed', 'tried', 'actually', '...",36,439,68
1,39293,Contrave,"""Contrave combines drugs that were used for al...",5-Mar-17,35,Weight Loss,9,contrave combines drugs that were used for alc...,"['work', 'want', 'contrave', 'know', 'obesity'...",62,782,143


In [32]:
drug_test_data["combine_name_use_review"] = drug_test_data.loc[:, ["name_of_drug", "use_case_for_drug", "review_by_patient_filter"]].fillna("").apply(lambda x: combine_name_use_review(x["name_of_drug"], x["use_case_for_drug"], x["review_by_patient_filter"]), axis=1)
drug_test_data.head(2)

,patient_id,name_of_drug,review_by_patient,drug_approved_by_UIC,number_of_times_prescribed,use_case_for_drug,effectiveness_rating,review_by_patient_filter,review_by_patient_unique_word,len_review_by_patient_unique_word,review_len,review_n_words,combine_name_use_review
0,163740,Mirtazapine,"""I&#039;ve tried a few antidepressants over th...",28-Feb-12,22,Depression,10,i 039 ve tried a few antidepressants over the...,"['insomnia', 'changed', 'tried', 'actually', '...",36,439,68,mirtazapine depression i 039 ve tried a few a...
1,39293,Contrave,"""Contrave combines drugs that were used for al...",5-Mar-17,35,Weight Loss,9,contrave combines drugs that were used for alc...,"['work', 'want', 'contrave', 'know', 'obesity'...",62,782,143,contrave weight loss contrave combines drugs t...


In [33]:
all_test_name_use_review = list(drug_test_data["combine_name_use_review"])
len(all_test_name_use_review)

10760

In [34]:
def find_vec(sent):
    vec = tokenizer.encode(sent)
    vec = tf.constant(vec)[None, :]
    return model1(vec)[0][0, 0, :].numpy()

distill_bert_embedding_test = np.zeros((len(all_test_name_use_review), 768))
def create_embadding_distil():
    global distill_bert_embedding_test
    embedding_dim = 768
    for i, sent in enumerate(all_test_name_use_review):
        embedding_vector = find_vec(sent[:2000])
        print(i, sent[0:20])
        if embedding_vector is not None:
            distill_bert_embedding[i] = embedding_vector
        else:
            print(i,"NNNNNNNNNNNNNNNNNNNN",  word)
    return distill_bert_embedding

In [35]:
if 'distill_bert_embedding_test.pkl' in os.listdir("../ix_word"):
    print("load")
    distill_bert_embedding_test = load_pickle('../ix_word/distill_bert_embedding_test.pkl')
else:
    distill_bert_embedding_test = create_embadding_distil()
    dump_pickle('../ix_word/distill_bert_embedding_test.pkl', distill_bert_embedding_test)
    print("dump")

load


In [36]:
combine_name_use_review_df_test = pd.DataFrame(distill_bert_embedding_test, columns=[f"review_{i}" for i in range(768)])
combine_name_use_review_df_test.head(2)

,review_0,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,...,review_758,review_759,review_760,review_761,review_762,review_763,review_764,review_765,review_766,review_767
0,-0.019849,0.173974,0.128156,-0.329652,-0.305466,-0.374780,0.513443,0.256115,-0.174848,-0.051410,...,-0.111515,-0.228544,-0.033203,-0.091765,0.380650,0.049244,-0.215225,-0.279196,0.303954,0.058522
1,0.079807,0.090857,0.170656,-0.152019,-0.202442,-0.290995,0.381226,0.454116,-0.339259,-0.183735,...,-0.072119,-0.301159,-0.060482,-0.068246,0.254681,0.206014,-0.216410,-0.261225,0.255563,0.014028


In [37]:
print("Number of features in preprocessed dataframe :", drug_test_data.shape[1])
print("Number of features in nlp dataframe :", combine_name_use_review_df_test.shape[1])

Number of features in preprocessed dataframe : 13
Number of features in nlp dataframe : 768


In [38]:
# storing the final features to csv file
if not os.path.isfile('../ix_word/final_features_test.csv'):
    combine_name_use_review_df_test['patient_id']=drug_test_data['patient_id']
    combine_name_use_review_df_test['effectiveness_rating']=drug_test_data['effectiveness_rating']
    combine_name_use_review_df_test['drug_approved_by_UIC']=drug_test_data['drug_approved_by_UIC']
    combine_name_use_review_df_test['number_of_times_prescribed']=drug_test_data['number_of_times_prescribed']
    combine_name_use_review_df_test['len_review_by_patient_unique_word']=drug_test_data['len_review_by_patient_unique_word']
    first_column = combine_name_use_review_df_test.pop('patient_id')
    combine_name_use_review_df_test.insert(0, 'patient_id', first_column)
    combine_name_use_review_df_test.to_csv('../ix_word/final_features_test.csv')

In [39]:
combine_name_use_review_df

,review_0,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,...,review_758,review_759,review_760,review_761,review_762,review_763,review_764,review_765,review_766,review_767
0,0.059691,0.008081,0.080178,-0.266999,-0.264136,-0.360968,0.646330,0.423532,-0.074173,-0.070809,...,-0.140880,-0.184657,0.068444,-0.305117,0.219165,-0.065599,-0.351725,-0.171581,0.362696,-0.022024
1,-0.477708,-0.197286,-0.113292,-0.019980,-0.573079,-0.220722,0.349210,-0.051921,-0.013572,-0.067297,...,0.134501,-0.330719,-0.052216,0.190429,0.398500,0.230648,-0.213165,-0.396574,0.421274,0.357504
2,0.142706,0.066007,-0.115109,-0.245073,-0.404206,-0.099233,0.558988,0.287499,0.084216,-0.071515,...,-0.149220,-0.202997,-0.084478,-0.328297,0.242339,-0.014993,-0.289103,-0.226220,0.362194,0.199670
3,-0.087916,0.229568,0.081001,-0.209838,0.004683,-0.293463,0.428451,0.510787,-0.121632,-0.229899,...,-0.079339,-0.173244,0.017691,-0.230129,0.202519,-0.255395,-0.274087,-0.205803,0.246486,0.253448
4,-0.271980,0.023836,0.053826,-0.313788,-0.439465,-0.367279,0.474098,0.236966,-0.100436,-0.254184,...,-0.057910,-0.430544,-0.003132,0.047696,0.365475,0.232395,-0.183236,-0.305313,0.196954,0.247114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32160,-0.025693,0.060552,0.028152,-0.303997,-0.072068,-0.020022,0.416418,0.524290,-0.104854,-0.238810,...,-0.049848,-0.298715,0.019632,-0.175059,0.215841,-0.022023,-0.240635,-0.156761,0.312285,0.353882
32161,0.017898,0.132657,0.272135,-0.005240,-0.158938,-0.226180,0.550534,0.561894,-0.256864,-0.135150,...,-0.156667,-0.338914,0.021950,-0.118178,0.221388,-0.073159,-0.322715,-0.527602,0.311378,0.064666
32162,-0.154145,0.190222,0.170894,-0.310281,-0.231704,-0.369669,0.512393,0.414607,-0.019496,-0.201162,...,-0.030458,-0.180127,-0.021214,-0.098297,0.323354,-0.077019,-0.263817,-0.255950,0.457359,0.134897
32163,0.004854,0.004234,0.149718,-0.234569,-0.357956,-0.022622,0.382827,0.521656,-0.264836,-0.200878,...,-0.127559,-0.177697,0.036091,-0.328201,0.209254,-0.001249,-0.181040,-0.206644,0.307512,0.279487


In [40]:
combine_name_use_review_df_test

,review_0,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,...,review_758,review_759,review_760,review_761,review_762,review_763,review_764,review_765,review_766,review_767
0,-0.019849,0.173974,0.128156,-0.329652,-0.305466,-0.374780,0.513443,0.256115,-0.174848,-0.051410,...,-0.111515,-0.228544,-0.033203,-0.091765,0.380650,0.049244,-0.215225,-0.279196,0.303954,0.058522
1,0.079807,0.090857,0.170656,-0.152019,-0.202442,-0.290995,0.381226,0.454116,-0.339259,-0.183735,...,-0.072119,-0.301159,-0.060482,-0.068246,0.254681,0.206014,-0.216410,-0.261225,0.255563,0.014028
2,-0.074708,-0.016995,0.414342,-0.308373,-0.060608,-0.190645,0.583799,0.645578,-0.059241,-0.300051,...,-0.157651,-0.016466,0.058117,-0.114096,0.126178,-0.066831,-0.213132,-0.483495,0.408229,0.157913
3,-0.109779,0.178474,0.281025,-0.213470,-0.211641,-0.360548,0.450201,0.481589,-0.108589,-0.165566,...,0.052805,-0.292624,-0.050574,-0.182317,0.256800,0.002749,-0.147731,-0.154975,0.328237,0.123911
4,0.033426,0.124495,0.069546,-0.365152,-0.168140,-0.293745,0.471172,0.675921,-0.193613,-0.444500,...,-0.118184,-0.282076,0.015641,-0.216246,0.355150,0.013335,-0.064397,-0.193974,0.426754,0.086814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32160,-0.025693,0.060552,0.028152,-0.303997,-0.072068,-0.020022,0.416418,0.524290,-0.104854,-0.238810,...,-0.049848,-0.298715,0.019632,-0.175059,0.215841,-0.022023,-0.240635,-0.156761,0.312285,0.353882
32161,0.017898,0.132657,0.272135,-0.005240,-0.158938,-0.226180,0.550534,0.561894,-0.256864,-0.135150,...,-0.156667,-0.338914,0.021950,-0.118178,0.221388,-0.073159,-0.322715,-0.527602,0.311378,0.064666
32162,-0.154145,0.190222,0.170894,-0.310281,-0.231704,-0.369669,0.512393,0.414607,-0.019496,-0.201162,...,-0.030458,-0.180127,-0.021214,-0.098297,0.323354,-0.077019,-0.263817,-0.255950,0.457359,0.134897
32163,0.004854,0.004234,0.149718,-0.234569,-0.357956,-0.022622,0.382827,0.521656,-0.264836,-0.200878,...,-0.127559,-0.177697,0.036091,-0.328201,0.209254,-0.001249,-0.181040,-0.206644,0.307512,0.279487


In [41]:
combine_name_use_review_df_test = combine_name_use_review_df_test.dropna()

In [42]:
combine_name_use_review_df_test.to_csv('../ix_word/final_features_test.csv')